# The Hypothesis

“Since the UK was one of the main countries that colonised the USA, and the UK is on the east side of
the USA there are more towns/cities with UK names on the east coast of the US rather than the west
coast”

## Data

https://github.com/apache/commons-csv/raw/master/src/test/resources/perf/worldcitiespop.txt.gz

lets get the imports out of the way

In [4]:
import pandas as pd

In [5]:
dataset = "/home/stormfield/scratch/DLG/worldcitiespop.txt"

In [11]:
dataset_df = pd.read_csv(dataset,encoding = "ISO-8859-1", low_memory=False)

the file wasn't UTF-8 encoded. The is 'ISO-8859-1' encoded (LATIN-1)

(I was about seek clarification regarding th encoding, but on closer examination of the actual use of the dataset in the apache commons, bench marking test class in github, it seems the file was infact 'ISO-8859-1'. Please refer [this](https://github.com/apache/commons-csv/blob/master/src/test/java/org/apache/commons/csv/CSVBenchmark.java) java file at line numbers 64 & 66)

## Data Exploration

Lets see what the data looks like

In [12]:
dataset_df.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,06,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,06,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,06,NaN,42.466667,1.500000
3,ad,aixirvall,Aixirvall,06,NaN,42.466667,1.500000
4,ad,aixovall,Aixovall,06,NaN,42.466667,1.483333


Lets verify the assumption of country names ie GB for United Kingdom & US for United States of America. [source](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes)

In [17]:
# united kingdom
'gb' in dataset_df['Country'].unique()

True

In [18]:
# USA
'us' in dataset_df['Country'].unique()

True

lets see what each of the data looks like

In [21]:
#uk_cities  
dataset_df.loc[dataset_df['Country'] == 'gb' ].head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
826690,gb,abberley,Abberley,Q4,NaN,52.300000,-2.366667
826691,gb,abberton,Abberton,F2,NaN,51.833333,0.916667
826692,gb,abberton,Abberton,F7,NaN,52.183333,-2.016667
826693,gb,abbess roding,Abbess Roding,E4,NaN,51.783333,0.266667
826694,gb,abbey-cwmhir,Abbey-Cwmhir,Y8,NaN,52.333333,-3.400000


In [30]:
#us_cities
dataset_df.loc[dataset_df['Country'] == 'us' ].head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
2532482,us,abanda,Abanda,AL,NaN,33.100833,-85.529722
2532483,us,abbeville,Abbeville,AL,NaN,31.571667,-85.250556
2532484,us,abbot springs,Abbot Springs,AL,NaN,33.360833,-86.481667
2532485,us,abel,Abel,AL,NaN,33.548611,-85.712500
2532486,us,abercrombie,Abercrombie,AL,NaN,32.848611,-87.165000


In [116]:
us = dataset_df.loc[dataset_df['Country'] == 'us' ].copy()

In [117]:
uk=  dataset_df.loc[dataset_df['Country'] == 'gb' ].copy()

In [118]:
us.duplicated('City').sum()

56025

In [119]:
uk.duplicated('City').sum()

1437

In [120]:
us.loc[us['City'] == 'london' ].head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude
2534705,us,london,London,AL,NaN,31.297500,-87.087778
2541129,us,london,London,AR,NaN,35.328889,-93.252778
2545390,us,london,London,CA,NaN,36.476111,-119.442222
2567874,us,london,London,IN,NaN,39.625556,-85.920278
2575484,us,london,London,KY,NaN,37.128889,-84.083333


Since the current task is see if the UK cities are there in US we need to only consider distinct cities in UK. 

But for US we need to consider all the cities since the same city name could be in more than one region. For example the UK city `London` is there in more than one region in the USA.
Hence they are valid for our analysis.

So for the UK data we will consider distinct city names

In [121]:
 uk.drop_duplicates(subset='City',inplace=True)